In [1]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd
import glob
from collections import Counter
import spacy
%store -r df


no stored variable or alias df


In [ ]:
"""# Get all CSV files in directory
json_files = glob.glob("data/*.csv")  # change it to the json file!!!!!!!!!!!!!!!!!!!!!!

# Print the list of CSV files with numbering
print("JSON Files:")
for i, file in enumerate(json_files, 1):
    print(f"{i}. {file}")"""

In [ ]:
"""# Read Json files and concat to a single DF
dfs = []
for f in json_files:
    try:
        df = pd.read_csv(f)
        print(f"Read {len(df)} rows from {f}")
        dfs.append(df)
    except pd.errors.EmptyDataError:
        print(f"Warning: {f} is empty")

# Concatenate DataFrames
df = pd.concat(dfs, ignore_index=True)"""

In [ ]:
# Print the first five rows of the 'job_description' column, showing about 120 characters
display(df["job_description"].str.slice(0, 120).head(5))

In [ ]:
# Extract text from all columns and concatenate
columns_to_concat = [
    "job_title",
    "company_name",
    "location",
    "job_link",
    "job_id",
    "job_description",
]
texts = " ".join(df[columns_to_concat].astype(str).apply(lambda x: " ".join(x), axis=1))

# Generate word cloud
wordcloud = WordCloud(width=900, height=380, colormap="summer").generate(texts)

# Plot word cloud
plt.figure(figsize=(16, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.title("\nSkills Word Cloud\n", size=18, color="#222222")
plt.axis("off")

# Set the background color of the figure to white
plt.gcf().patch.set_facecolor("white")

# Adjust layout to center the plot
plt.tight_layout()

plt.show()

In [ ]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Tokenize the text
doc = nlp(texts)

# Perform frequency analysis
word_freq = Counter(token.text for token in doc)

# Display the most common words
print(word_freq.most_common())

# Display the top N most common words
top_n = 20
print(f"Top {top_n} Most Common Words:")
for word, count in word_freq.most_common(top_n):
    print(f"{word}: {count}")

In [ ]:
# Display the top N most common words
top_n = 25
top_words, top_frequencies = zip(*word_freq.most_common(top_n))

# Normalize frequencies to use as sizes for dots
sizes = [freq * 10 for freq in top_frequencies]

# Create a scatter plot
plt.figure(figsize=(16, 4.5))
ax = plt.gca()

# Set the background color of the entire figure
fig = plt.gcf()
fig.patch.set_facecolor("white")

# Plot scatter points
plt.scatter(top_words, top_frequencies, alpha=0.7, edgecolors="w", marker="o")

# Style the plot
plt.style.use("classic")
ax.grid(True)

from matplotlib.colors import LogNorm

# Create a scatter plot with different colors and sizes
scatter = ax.scatter(
    top_words,
    top_frequencies,
    c=top_frequencies,
    cmap="summer",
    s=sizes,
    alpha=0.7,
    edgecolors="w",
    marker="o",
    norm=LogNorm(),
)

# Set plot title with background color
plt.title("\nTop 25 Word Frequency Scatter Plot\n", size=18, color="#222222")

# Label the axes
plt.xlabel("\nWords\n", size=14, color="#222222")
plt.ylabel("\nFrequency\n", size=14, color="#222222")

# Rotate x-axis labels for better readability
plt.xticks(rotation=35, ha="right", color="#333333")
plt.yticks(color="#333333")

# Remove top and right spines
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

# Set the left and bottom spines to grey color
ax.spines["left"].set_color("grey")
ax.spines["bottom"].set_color("grey")

# Display the plot
plt.show()

In [ ]:
import plotly.express as px

# Assuming 'df' is your DataFrame with frequency data
fig = px.scatter(
    df,
    x="Skill",
    y="Frequency",
    color="Cluster",
    size="Frequency",
    hover_data=["Reference Text"],
)

# To display the interactive plot in Jupyter Notebook or Jupyter Lab
fig.show()